Importação das Bibliotecas

In [1]:
import os
import zipfile
import shutil
import cv2
import numpy as np
import pandas as pd
from PIL import Image
import glob

Diretório das Imagens e Saídas de .CSV e DataFrame

In [2]:
# Treino Sentinel-2
origem = "/home/cristiano/Imagens/imagens_originais/Sentinel-2/dados/"
diretorio_imagens = "/home/cristiano/Imagens/imagens_originais/Sentinel-2/dados_processados/"
# Teste Sentinel-2
# origem = "/home/cristiano/Imagens/imagens_originais/Sentinel-2/teste/"
# diretorio_imagens = "/home/cristiano/Imagens/imagens_originais/Sentinel-2/teste_processado/"
# Teste Landsat-8
#origem = "/home/cristiano/Imagens/imagens_originais/Landsat-8/teste/"
#diretorio_imagens = "/home/cristiano/Imagens/imagens_originais/Landsat-8/teste_processado/"

Extrair as imagens para o diretório "treino_processado" e adicionando um número para diferenciar as imagens de mesmo nome.

In [3]:
numero_imagem = 1

# Verifica se o diretório de destino existe, caso contrário, cria-o
if not os.path.exists(diretorio_imagens):
    os.makedirs(diretorio_imagens)

# Percorre todas as pastas compactadas no diretório de origem
for pasta_compactada in os.listdir(origem):
    caminho_pasta_compactada = os.path.join(origem, pasta_compactada)

    # Verifica se o item é uma pasta compactada
    if zipfile.is_zipfile(caminho_pasta_compactada):
        # Extrai o conteúdo do arquivo compactado para uma pasta temporária
        pasta_temporaria = os.path.join(diretorio_imagens, "temp")
        with zipfile.ZipFile(caminho_pasta_compactada, 'r') as zip_ref:
            zip_ref.extractall(pasta_temporaria)

        # Percorre todas as imagens na pasta temporária
        for nome_imagem in os.listdir(pasta_temporaria):
            caminho_imagem = os.path.join(pasta_temporaria, nome_imagem)

            # Verifica se o item é um arquivo de imagem
            if os.path.isfile(caminho_imagem):
                # Define o novo nome da imagem com o número no início
                novo_nome_imagem = f"{numero_imagem}_{nome_imagem}"

                # Move a imagem para o diretório de destino com o novo nome
                caminho_destino_imagem = os.path.join(diretorio_imagens, novo_nome_imagem)
                shutil.move(caminho_imagem, caminho_destino_imagem)

                # Incrementa o número para a próxima imagem
                numero_imagem += 1

        # Remove a pasta temporária após processar todas as imagens
        shutil.rmtree(pasta_temporaria)

print("Descompactação concluída.")

Descompactação concluída.


Criação das máscaras. 

In [4]:
def modificar_nome_arquivo(nome_arquivo):
    # Remover a sequência "_(Raw)"
    nome_arquivo_modificado = nome_arquivo.replace("B12_(Raw)", "")
    
    # Substituir "Wildf" por "F"
    #nome_arquivo_modificado = nome_arquivo_modificado.replace("Wildf", "F")
    
    return nome_arquivo_modificado


def limiar(diretorio):
    # Percorre todas as imagens no diretório
    for nome_arquivo in os.listdir(diretorio):
        if nome_arquivo.endswith("B12_(Raw).tiff"):
            # Limpar o nome do arquivo
            nome_arquivo_limpo = modificar_nome_arquivo(nome_arquivo)

            # Carrega a imagem
            caminho_arquivo = os.path.join(diretorio, nome_arquivo)
            imagem = cv2.imread(caminho_arquivo)

            # Aplicar o limiar
            limiar = 180
            _, thresh = cv2.threshold(imagem, limiar, 1, cv2.THRESH_BINARY)

            # Obter o nome do arquivo sem a extensão
            nome_sem_extensao = os.path.splitext(nome_arquivo_limpo)[0]

            # Salvar a imagem limiarizada com "_Mask" antes da extensão no mesmo diretório
            nome_mask = nome_sem_extensao + "Mask.tiff"
            caminho_mask = os.path.join(diretorio, nome_mask)
            cv2.imwrite(caminho_mask, thresh)

    # Libera a memória das imagens
    cv2.destroyAllWindows()

limiar(diretorio_imagens)


Recorte das imagens em 64x64

In [5]:
def recortar_imagens(diretorio, tamanho_recorte):
    # Percorre todas as imagens no diretório
    for nome_arquivo in os.listdir(diretorio):
        if nome_arquivo.endswith(".tiff"):
            # Carrega a imagem
            caminho_arquivo = os.path.join(diretorio, nome_arquivo)
            imagem = cv2.imread(caminho_arquivo)
            
            # Obtém as dimensões da imagem
            altura, largura, _ = imagem.shape
            
            # Calcula o número de recortes horizontais e verticais
            num_recortes_horizontais = largura // tamanho_recorte
            num_recortes_verticais = altura // tamanho_recorte
            
            # Realiza o recorte e salva as imagens resultantes
            for i in range(num_recortes_horizontais):
                for j in range(num_recortes_verticais):
                    # Define as coordenadas do retângulo de recorte
                    esquerda = i * tamanho_recorte
                    superior = j * tamanho_recorte
                    direita = (i + 1) * tamanho_recorte
                    inferior = (j + 1) * tamanho_recorte
                    
                    # Realiza o recorte
                    recorte = imagem[superior:inferior, esquerda:direita]
                    
                    # Salva o recorte em um novo arquivo
                    nome_recorte = f"recorte_{i}_{j}_{nome_arquivo}"
                    caminho_recorte = os.path.join(diretorio, nome_recorte)
                    cv2.imwrite(caminho_recorte, recorte)
                    
            # Remove a imagem original
            os.remove(caminho_arquivo)
            
    # Libera a memória das imagens
    cv2.destroyAllWindows()

tamanho_recorte = 64
recortar_imagens(diretorio_imagens, tamanho_recorte)


Transformar imagens para .CSV

In [6]:
def tiff_to_csv(image_directory):
    # Listar todos os arquivos no diretório
    file_list = os.listdir(image_directory)

    # Filtrar apenas os arquivos com extensão .tiff
    tiff_files = [file for file in file_list if file.endswith('.tiff')]

    # Percorrer cada imagem .tiff
    for tiff_file in tiff_files:
        # Obter o caminho completo do arquivo .tiff
        image_path = os.path.join(image_directory, tiff_file)

        # Abrir a imagem .tiff
        image = Image.open(image_path)

        if 'Wildfires' not in tiff_file:
            # Converter a imagem para escala de cinza
            gray_image = image.convert("L")

            # Converter a imagem em escala de cinza para um array NumPy
            gray_array = np.array(gray_image)
            # Aplainar o array para um vetor 1D
            csv_data = gray_array.flatten()
        else:
            csv_data = []
            for y in range(image.height):
                for x in range(image.width):
                    # Extrair os valores de pixel desejados
                    pixel_value = image.getpixel((x, y))
                    csv_data.append(pixel_value)
        

        # Criar um DataFrame a partir dos dados CSV
        df = pd.DataFrame([csv_data])

        # Identificar o tipo de imagem com base no nome do arquivo
        if 'B11_(Raw)' in tiff_file or 'B12_(Raw)' in tiff_file:
            csv_filename = 'bandas_sentinel2.csv'
        elif 'Wildfires' in tiff_file:
            csv_filename = 'wildfires.csv'
        elif 'Mask' in tiff_file:
            csv_filename = 'mask.csv'
        elif 'B06_(Raw)' in tiff_file or 'B07_(Raw)' in tiff_file:
            csv_filename = 'mask_bandasLandsat8.csv'
        elif 'Custom_script' in tiff_file:
            csv_filename = 'mask_custom.csv'
        else:
            print(f"Arquivo não correspondente: {tiff_file}")

        # Caminho completo do arquivo .csv
        csv_path = os.path.join(image_directory, csv_filename)

        # Escrever o DataFrame para o arquivo .csv
        df.to_csv(csv_path, index=False, header=False,mode='a')

        print(f"Conversão concluída para {tiff_file}. Arquivo CSV gerado: {csv_filename}")

tiff_to_csv(diretorio_imagens)


Conversão concluída para recorte_1_1_127_2022-08-26-00:00_2022-08-26-23:59_Sentinel-2_L2A_Wildfires.tiff. Arquivo CSV gerado: wildfires.csv
Conversão concluída para recorte_7_3_98_2021-08-08-00:00_2021-08-08-23:59_Sentinel-2_L2A_B12_(Raw).tiff. Arquivo CSV gerado: bandas_sentinel2.csv
Conversão concluída para recorte_5_1_98_2021-08-08-00:00_2021-08-08-23:59_Sentinel-2_L2A_Mask.tiff. Arquivo CSV gerado: mask.csv
Conversão concluída para recorte_1_3_97_2021-08-08-00:00_2021-08-08-23:59_Sentinel-2_L2A_Wildfires.tiff. Arquivo CSV gerado: wildfires.csv
Conversão concluída para recorte_9_1_64_2022-01-30-00:00_2022-01-30-23:59_Sentinel-2_L2A_B11_(Raw).tiff. Arquivo CSV gerado: bandas_sentinel2.csv
Conversão concluída para recorte_2_0_15_2023-02-07-00:00_2023-02-07-23:59_Sentinel-2_L2A_B12_(Raw).tiff. Arquivo CSV gerado: bandas_sentinel2.csv
Conversão concluída para recorte_7_13_98_2021-08-08-00:00_2021-08-08-23:59_Sentinel-2_L2A_Mask.tiff. Arquivo CSV gerado: mask.csv
Conversão concluída para

FileNotFoundError: [Errno 2] No such file or directory: '/home/cristiano/Imagens/imagens_originais/Sentinel-2/dados_processados/recorte_16_3_85_2021-09-08-00:00_2021-09-08-23:59_Sentinel-2_L2A_B12_(Raw).tiff'

Criação dos DataFrame

In [ ]:
def csv_to_dataframe(diretorio_imagens):
    # Dicionário para armazenar os DataFrames de cada arquivo CSV
    dfs = {}

    # Percorrer todos os arquivos no diretório
    for filename in os.listdir(diretorio_imagens):
        if filename.endswith('.csv'):
            # Obter o caminho completo do arquivo CSV
            filepath = os.path.join(diretorio_imagens, filename)

            # Ler o arquivo CSV e criar um DataFrame
            df = pd.read_csv(filepath)

            # Adicionar o DataFrame ao dicionário usando o nome do arquivo como chave
            dfs[filename] = df

            # Criar o novo nome de arquivo para o DataFrame
            new_filename = f"df_{os.path.splitext(filename)[0]}.csv"

            # Salvar o DataFrame em um arquivo CSV no mesmo diretório com o novo nome
            df.to_csv(os.path.join(diretorio_imagens, new_filename), index=False, header=list(range(1, len(df.columns)+1)))

            # Imprimir o nome do DataFrame gerado
            print(f"DataFrame gerado para o arquivo {new_filename}")

    return dfs

dfs = csv_to_dataframe(diretorio_imagens)


DataFrame gerado para o arquivo df_bandas_sentinel2.csv
DataFrame gerado para o arquivo df_mask.csv
DataFrame gerado para o arquivo df_wildfires.csv


In [ ]:
# Padrão para os arquivos CSV
padrao_arquivos = "*.csv"

# Lista de caminhos para os arquivos CSV
caminhos_df = glob.glob(f"{diretorio_imagens}/{padrao_arquivos}")

# Loop para ler e imprimir o cabeçalho de cada DataFrame
for caminho in caminhos_df:
    df = pd.read_csv(caminho)
    print(f"Cabeçalho do DataFrame {caminho}:")
    print(df.shape)
    print(df.head())
    print()  # Linha em branco para separar os resultados


Cabeçalho do DataFrame /home/cristiano/Imagens/imagens_originais/Sentinel-2/imagens_processado/df_bandas_sentinel2.csv:
(6819, 4096)
    1   2   3   4   5   6   7   8   9  10  ...  4087  4088  4089  4090  4091  \
0  63  63  63  62  58  55  53  52  52  52  ...    80    79    78    79    79   
1  46  49  54  54  54  54  54  55  56  56  ...    46    47    48    48    47   
2  93  92  91  90  89  89  90  91  92  92  ...    49    50    51    51    53   
3  32  34  36  41  45  47  47  44  37  30  ...    16    16    17    18    19   
4  63  63  62  61  60  60  60  59  60  62  ...    73    72    70    69    67   

   4092  4093  4094  4095  4096  
0    79    79    79    78    78  
1    45    45    48    49    49  
2    53    53    53    54    55  
3    21    23    25    25    22  
4    66    66    66    66    66  

[5 rows x 4096 columns]

Cabeçalho do DataFrame /home/cristiano/Imagens/imagens_originais/Sentinel-2/imagens_processado/bandas_sentinel2.csv:
(6819, 4096)
   42  41  39  38  36  34 

In [ ]:
train = pd.read_csv("/home/cristiano/Imagens/imagens_originais/Sentinel-2/imagens_processado/df_bandas_sentinel2.csv")
print(train.shape)
train.head()

(6819, 4096)


,1,2,3,4,5,6,7,8,9,10,...,4087,4088,4089,4090,4091,4092,4093,4094,4095,4096
0,63,63,63,62,58,55,53,52,52,52,...,80,79,78,79,79,79,79,79,78,78
1,46,49,54,54,54,54,54,55,56,56,...,46,47,48,48,47,45,45,48,49,49
2,93,92,91,90,89,89,90,91,92,92,...,49,50,51,51,53,53,53,53,54,55
3,32,34,36,41,45,47,47,44,37,30,...,16,16,17,18,19,21,23,25,25,22
4,63,63,62,61,60,60,60,59,60,62,...,73,72,70,69,67,66,66,66,66,66


In [ ]:
train = pd.read_csv("/home/cristiano/Imagens/imagens_originais/Sentinel-2/teste_processado/df_wildfires.csv")
print(train.shape)
train.head()

In [ ]:
train = pd.read_csv("/home/cristiano/Imagens/imagens_originais/Sentinel-2/teste_processado/df_mask_banda12.csv")
print(train.shape)
train.head()

In [ ]:
train = pd.read_csv("/home/cristiano/Imagens/imagens_originais/Landsat-8/teste_processado/df_mask_banda06.csv")
print(train.shape)
train.head()